In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Maza\\Desktop\\Pinecone_pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    read_data_dir: Path
    STATUS_FILE: str
    SCHEMA: dict



In [5]:
from vector_db_pipeline.constants import *
from vector_db_pipeline.utils.common import read_yaml, create_directories
from vector_db_pipeline import logger
import pandas as pd

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            read_data_dir=config.read_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            SCHEMA=schema
        )

        return data_validation_config
    

In [8]:
import os
from vector_db_pipeline import logger
import pandas as pd

In [35]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        self.data = pd.read_json(self.config.read_data_dir, orient='records')


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            
            all_cols = list(self.data.columns)
            

            all_schema = self.config.SCHEMA
            all_schema['id'] = 'str'
            all_schema['values'] = 'list'
            print(all_schema)
            print(all_cols)

            
            for col in all_cols:
                if col not in all_schema.keys():
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"All columns present in data: {validation_status}\n")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"All columns present in data: {validation_status}\n")

            return logger.info(f"All columns present in data: {validation_status}")
        
        except Exception as e:
            raise e
        
    def validate_unique_index(self)-> bool:
        try:
            
            unique_id = len(self.data.id.unique()) == len(self.data)
            if unique_id:
             
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Unique ids : {unique_id}\n")
            else:
               
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Unique ids : {unique_id}\n")
            return logger.info(f"Unique ids : {unique_id}")
        
        except Exception as e:
            raise e
        

    def get_column_types(self):
        """
        Get the types of values in each column of the DataFrame.

        Args:
            df (DataFrame): The input DataFrame.

        Returns:
            dict: A dictionary where keys are column names and values are lists containing the types of values present in each column.
        """
        
        try:
        
            column_types = {}
            how_many_types ={}
            for column in self.data.columns:
                # Create a set of unique types for values in the column
                types = set(type(value) for value in self.data[column])
                # Convert the set to a list for easier handling
                types_list = list(types)

                
                column_types[column] = types_list
                how_many_types[column]=len(types_list)
            
            id_type = column_types['id'][0]==str
            vector_type = column_types['values'][0]==list

            values_type = self.data['values'][0][0]==float
            types_set = set(type(elem) for elem in self.data['values'][0])
            print(types_set)
            if not id_type :
                raise TypeError("id not a string")
            elif not vector_type:
                raise TypeError("values not a list")
            elif not values_type:
                raise TypeError("vector values not float")
            
            unique_types = all(value == 1 for value in how_many_types.values())

            if not unique_types:
                raise TypeError("several types in data")

            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Data types are correct\n")

            return logger.info(f"Data types are valid")
        except Exception as e:
            raise e

        



In [36]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_unique_index()
    data_validation.get_column_types()
except Exception as e:
    raise e

[2024-04-14 15:01:43,709: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-04-14 15:01:43,713: INFO: common: yaml file: schema.yaml loaded successfully:]
[2024-04-14 15:01:43,715: INFO: common: Directory already exists: artifacts/data_validation:]
{'date_scraped_timestamp': 'int', 'host': 'str', 'page_title': 'str', 'text': 'str', 'url': 'str', 'id': 'str', 'values': 'list'}
['id', 'values', 'text', 'host', 'page_title', 'url']
[2024-04-14 15:01:43,754: INFO: 4179254809: All columns present in data: True:]
[2024-04-14 15:01:43,754: INFO: 4179254809: Unique ids : True:]
[0.0073613964, -0.0216906355, 0.0233809822, -0.0282842783, -0.0153937706, 0.026761675800000002, 0.0323617577, 0.0044129669, -0.018297037500000002, -0.0200648051, 0.0292907456, 0.0337811343, -0.0081678603, -0.0307875412, -0.0090581954, 0.014245367, 0.027432654600000002, -0.000910497, -0.013167932, -0.0306068926, -0.017277669500000002, -0.0198970609, 0.0085743174, -0.0017193797, -0.0173679929, 0.0001

TypeError: vector values not float